In [ ]:
import numpy as np
import directlfq.benchmarking as lfq_benchmark

class NormalizationVisualization():
    def __init__(self, ion_dataframe, ax, cumulative = False, histtype = 'step', density = True, bins = 150):
        self._prepared_dataframe = DataframePreparerNormalizationPlot(ion_dataframe)
        self._ax = ax
        self._histconfig = lfq_benchmark.HistPlotConfig(cumulative, histtype, density, bins)
    
    def plot_shift_histogram(self):
        for column in self._prepared_dataframe.reference_normalized_dataframe.columns:
            column_intensities = self._prepared_dataframe.reference_normalized_dataframe[column]
            reference_intensities = self._prepared_dataframe.reference_normalized_dataframe[self._prepared_dataframe.reference_column]
            fcs_to_reference = column_intensities - reference_intensities
            self._ax.hist(fcs_to_reference, cumulative = self._histconfig.cumulative, histtype = self._histconfig.histtype, density = self._histconfig.density, bins = self._histconfig.bins)


class HistPlotConfig():
    def __init__(self,  cumulative, histtype, density, bins):
        self.cumulative = cumulative
        self.histtype = histtype
        self.density = density
        self.bins = bins

class DataframePreparerNormalizationPlot():
    def __init__(self, ion_dataframe):
        self._ion_dataframe = ion_dataframe
        self.reference_column = None
        self.reference_normalized_dataframe = None

    def _define_reference_normalized_dataframe(self):
        self._select_column_with_most_valid_values_as_reference()
    
    def _select_column_with_most_valid_values_as_reference(self):
        sorted_columnnames = sorted(self._ion_dataframe.columns, lambda x : self._get_num_valid_values(x), reverse=True)
        self.reference_column = sorted_columnnames[0]
    
    def _get_num_valid_values(self, column_name):
        values = self._ion_dataframe[column_name].to_numpy()
        return sum(~np.isnan(values))

    def _subtract_reference_from_ion_dataframe(self):
        self.reference_normalized_dataframe = self._ion_dataframe - self._ion_dataframe[self.reference_column]
        self.reference_normalized_dataframe.drop(labels = self.reference_column, axis = 1)


